# Setting up a baseline model for benchmarking
The general architecture of the model will be using two indermediate models to predict the onblock and offblock delays which will be ultimately added to the scheduled times: next_landing_time = last_landing_time + onblock_sched + onblock_delay + offblock_sched + offblock_delay

Two simple baseline models will be established to predict the consecuetivee landing times of a given connection chain:
1. Predicting delays as means of delays
1. Random forests regression

In [14]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor


In [15]:
# Adjust settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
%matplotlib inline

In [16]:
# Load preprocessed dataset

data_path = '../data/processed/'
df = pd.read_pickle(os.path.join(data_path, 'final.pkl'))

In [17]:
df.head()

,flt_ac_reg,ground_delay,flt_ac_type,flt_tt,flt_sched_tt,block_delay,routing,sched_gt,act_gt,cp_count,ca_count,cc_cp_ca,cc_count,day_of_week,hour_of_day_dep,hour_of_day_arr,cc_types
9,ECLBAX,25.0,320,0.0,0.0,16.0,New Jessica_East Carmen,95.0,94.0,2,4,none,0,5,3,6,0
117,ECLBAX,15.0,320,60.0,60.0,6.0,East Carmen_South Nathaniel,75.0,120.0,2,4,both,6,5,8,10,3
194,ECLBAX,51.0,320,68.0,75.0,37.0,South Nathaniel_East Carmen,80.0,86.0,2,4,none,0,5,11,14,1
268,ECLBAX,43.0,320,64.0,70.0,32.0,East Carmen_Joneshaven,50.0,32.0,2,4,both,6,5,15,17,3
416,ECLBAX,20.0,320,73.0,45.0,22.0,East Carmen_Joneshaven,45.0,38.0,2,4,none,0,6,6,8,0


In [18]:
# One-hot encode all catgorical variables
df_one_hot = pd.get_dummies(df, drop_first=True)
df_one_hot.dropna(axis=0, how='any', inplace=True)

In [19]:
df_one_hot.head()

,ground_delay,flt_tt,flt_sched_tt,block_delay,sched_gt,act_gt,flt_ac_reg_ECLBBX,flt_ac_reg_ECLBCX,flt_ac_reg_ECLBDX,flt_ac_reg_ECLBEX,flt_ac_reg_ECLBFX,flt_ac_reg_ECLBIX,flt_ac_reg_ECLBJX,flt_ac_reg_ECLBKX,flt_ac_reg_ECLBLX,flt_ac_reg_ECLBMX,flt_ac_reg_ECLBNX,flt_ac_reg_ECLBOX,flt_ac_reg_ECLBPX,flt_ac_reg_ECLBQX,flt_ac_reg_ECLBRX,flt_ac_reg_ECLBSX,flt_ac_reg_ECLBTX,flt_ac_reg_ECLBUX,flt_ac_reg_ECLBVX,flt_ac_reg_ECLBWX,flt_ac_reg_ECLBXX,flt_ac_reg_ECLBYX,flt_ac_reg_ECLBZX,flt_ac_reg_ECLDAX,flt_ac_reg_ECLDBX,flt_ac_reg_ECLDCX,flt_ac_reg_ECLDDX,flt_ac_reg_ECLDEX,flt_ac_reg_ECLDFX,flt_ac_reg_ECLDGX,flt_ac_reg_ECLGAX,flt_ac_reg_ECLGBX,flt_ac_reg_ECLGCX,flt_ac_reg_ECLGEX,flt_ac_reg_ECLGFX,flt_ac_reg_ECLGGX,flt_ac_reg_ECLGHX,flt_ac_reg_ECLGIX,flt_ac_reg_ECLGJX,flt_ac_reg_ECLGKX,flt_ac_reg_ECLGLX,flt_ac_reg_ECLGMX,flt_ac_reg_ECLGNX,flt_ac_reg_ECLGOX,flt_ac_reg_ECLGPX,flt_ac_reg_ECLGQX,flt_ac_reg_ECLGRX,flt_ac_reg_ECLWAX,flt_ac_reg_ECLWBX,flt_ac_reg_ECLWCX,flt_ac_reg_ECLWDX,flt_ac_reg_ECLWEX,flt_ac_reg_ECLWFX,flt_ac_reg_ECLWGX,flt_ac_reg_ECLWHX,flt_ac_reg_ECLWIX,flt_ac_reg_ECLWJX,flt_ac_reg_ECLWKX,flt_ac_reg_ECLWLX,flt_ac_reg_ECLWMX,flt_ac_reg_ECLWNX,flt_ac_reg_ECLWOX,flt_ac_reg_ECLWPX,flt_ac_reg_ECLWQX,flt_ac_reg_ECLXAX,flt_ac_reg_ECLXBX,flt_ac_reg_ECLXCX,flt_ac_reg_ECLXDX,flt_ac_reg_ECLXEX,flt_ac_type_DH4,flt_ac_type_E95,routing_Aaronbury_North Stephenville,routing_Aaronbury_Rowefurt,routing_Aaronbury_Vaughnberg,routing_Aaronbury_Yoderburgh,routing_Castroville_East Carmen,routing_Christophershire_East Carmen,routing_Coleland_East Carmen,routing_Davidtown_East Carmen,routing_Dawsonhaven_East Allisontown,routing_Dawsonhaven_East Carmen,routing_Desireeton_East Carmen,routing_Dudleyton_East Carmen,routing_East Allisontown_Aaronbury,routing_East Allisontown_Dawsonhaven,routing_East Allisontown_East Carmen,routing_East Allisontown_Lake Gregory,routing_East Allisontown_Lake Lawrencechester,routing_East Allisontown_Reneemouth,routing_East Allisontown_Vaughnberg,routing_East Allisontown_Weeksbury,routing_East Allisontown_Yoderburgh,routing_East Carmen_Barnesmouth,routing_East Carmen_Castroville,routing_East Carmen_Christophershire,routing_East Carmen_Coleland,routing_East Carmen_Collinsshire,routing_East Carmen_Davidtown,routing_East Carmen_Dawsonhaven,routing_East Carmen_Desireeton,routing_East Carmen_Dudleyton,routing_East Carmen_East Allisontown,routing_East Carmen_East Elizabethside,routing_East Carmen_East Holly,routing_East Carmen_East Jasonmouth,routing_East Carmen_East Latashaview,routing_East Carmen_East Lisafurt,routing_East Carmen_East Melindachester,routing_East Carmen_East Melissaberg,routing_East Carmen_East Michael,routing_East Carmen_East Toddchester,routing_East Carmen_Ellisside,routing_East Carmen_Erichaven,routing_East Carmen_Fisherhaven,routing_East Carmen_Garrettland,routing_East Carmen_Gilbertland,routing_East Carmen_Haynesside,routing_East Carmen_Jamesview,routing_East Carmen_Jenkinsside,routing_East Carmen_Johnathanberg,routing_East Carmen_Joneshaven,routing_East Carmen_Juliemouth,routing_East Carmen_Kaylashire,routing_East Carmen_Keithberg,routing_East Carmen_Kennethfort,routing_East Carmen_Kristashire,routing_East Carmen_Lake Danielburgh,routing_East Carmen_Lake David,routing_East Carmen_Lake Gerald,routing_East Carmen_Lake Gregory,routing_East Carmen_Lake Kevin,routing_East Carmen_Lake Lawrencechester,routing_East Carmen_Lake Richardberg,routing_East Carmen_Lake Sarahview,routing_East Carmen_Lake Williammouth,routing_East Carmen_Lambertport,routing_East Carmen_Lisamouth,routing_East Carmen_Lloydtown,routing_East Carmen_Marioborough,routing_East Carmen_Maryland,routing_East Carmen_Matthewland,routing_East Carmen_Mezaport,routing_East Carmen_Michaelfurt,routing_East Carmen_Millerborough,routing_East Carmen_Mooretown,routing_East Carmen_Navarroville,routing_East Carmen_New Amy,routing_East Carmen_New Bobby,routing_East Carmen_New Harrytown,routing_East Carmen_New Jessica,routing_East Carmen_New Johnhaven,routing_East C

In [20]:
# Generate train/test splits for both intermediate models

X_train_offblock, X_test_offblock, y_train_offblock, y_test_offblock = train_test_split(df_one_hot.drop(['block_delay'], axis=1), df_one_hot['block_delay'], test_size=0.33, random_state=42)
X_train_onblock, X_test_onblock, y_train_onblock, y_test_onblock = train_test_split(df_one_hot.drop(['ground_delay'], axis=1), df_one_hot['ground_delay'], test_size=0.33, random_state=42)

In [21]:
# Implement first baseline model as mean of delays
dummy_regr_off = DummyRegressor(strategy='mean')
dummy_regr_off.fit(X_train_offblock, y_train_offblock)
dummy_regr_off.predict(X_test_offblock)

dummy_regr_on = DummyRegressor(strategy='mean')
dummy_regr_on.fit(X_train_onblock, y_train_onblock)
dummy_regr_on.predict(X_test_onblock)

array([21.04473142, 21.04473142, 21.04473142, ..., 21.04473142,
       21.04473142, 21.04473142])

In [22]:
r2_offblock = dummy_regr_off.score(X_test_offblock, y_test_offblock)
rmse_offblock = np.sqrt(mean_squared_error(y_test_offblock, dummy_regr_off.predict(X_test_offblock)))
print('The r^2 for offblock time is ' + str(round(r2_offblock, 4)))
print('The RMSE for offblock time is ' + str(round(rmse_offblock, 2)) + 'minutes.')
print('\n')

r2_onblock = dummy_regr_on.score(X_test_onblock, y_test_onblock)
rmse_onblock = np.sqrt(mean_squared_error(y_test_onblock, dummy_regr_on.predict(X_test_onblock)))
print('The r^2 for onblock time is ' + str(round(r2_onblock, 4)))
print('The RMSE for onblock time is ' + str(round(rmse_onblock, 2)) + 'minutes.')
print('\n')

The r^2 for offblock time is -0.0008
The RMSE for offblock time is 19.26minutes.


The r^2 for onblock time is -0.0006
The RMSE for onblock time is 18.08minutes.




In [23]:
# Implement second baseline model as random forest regression
rand_for_off = RandomForestRegressor(n_estimators = 10, random_state=42)
rand_for_off.fit(X_train_offblock, y_train_offblock)
Y_predict_off = rand_for_off.predict(X_test_offblock)

rand_for_on = RandomForestRegressor(n_estimators = 10, random_state=42)
rand_for_on.fit(X_train_onblock, y_train_onblock)
rand_for_on.predict(X_test_onblock)

array([-6.6, 17.3, 56.9, ..., 20.3,  1.3, 18.4])

In [24]:
r2_offblock = r2_score(y_test_offblock, Y_predict_off)
rmse_offblock = mean_squared_error(y_test_offblock, Y_predict_off, squared=False)
print('The r^2 for offblock time is ' + str(round(r2_offblock, 4)))
print('The RMSE for offblock time is ' + str(round(rmse_offblock, 2)) + 'minutes.')
print('\n')

r2_onblock = rand_for_on.score(X_test_onblock, y_test_onblock)
rmse_onblock = np.sqrt(mean_squared_error(y_test_onblock, rand_for_on.predict(X_test_onblock)))
print('The r^2 for onblock time is ' + str(round(r2_onblock, 4)))
print('The RMSE for onblock time is ' + str(round(rmse_onblock, 2)) + 'minutes.')
print('\n')

The r^2 for offblock time is 0.8967
The RMSE for offblock time is 6.19minutes.


The r^2 for onblock time is 0.9091
The RMSE for onblock time is 5.45minutes.


